<div class="alert alert-block alert-success">
    <b>Universidad Rey Juan Carlos de Madrid.</b>  Biomedical Engineering
</div>

# Optimization
### Linear Integer Programming for the Home Helath Care Problem

<div class="alert alert-block alert-info"> Pablo Laso Mielgo </div> 

In this Pyhton code I try to recreate a simplified version of the work performed by some other students in this
[article](https://link.springer.com/chapter/10.1007/978-3-642-28115-0_14). We'll count with:
* 3 nurses
* 2 patients <p>
<br>
* maximum/minimum workload
* qualifications
* distances
* patients needs
* treatment duration <br>
...

 ______________


<font color='red'>**Problem Statement:**</font>

<div class="alert alert-block alert-danger">
<b>Remark:</b> Choosing 3 patients and all possible paths would make for too many different variables, for each nurse: 
     $$\sum_{k,i,j = 1}^{3} load_{kij}$$
    <br> Due to this extremely large number of variables I could not manage, it was decided to reduce the number to just 2 patients.
</div>

---------
As stated in the paper given to us by Prof.Luis Ángel Calvo Pascual, *Planners in home health care services must then
establish a daily visit plan which specifies [...] which nurse should be assigned to it [the visit]*. This shall be our purpose. <p>
    
---
<br> This code tries to tackle several problems in one by establishing the most convienient care plan by the nurses taking into consideration the availability of the patients. For this:
* First, we try to minimize the time it takes to attend all patients. This means that our **objective function** will be:
$$ minimize: \sum_{k,i,j} load_{kij}*y_{kij}\quad \forall k  \quad{.....\quad(0)} $$ <br>
, where k is the nurse, I the first patient visited and j, the last patient visited.<p>
    * Also, we want to specify some **constraints** (bounds) to maintain consistency in our code. That is, we define, simililarly to the *Shortest Path Problem* seen in class, the following constraints: 
$$ \sum_{k,i,j} y_{kij}\quad =\quad Need_{patient_1} \quad{.....\quad(1)}$$ 
$$ \sum_{k,i,j} y_{kij}\quad =\quad Need_{patient_2} \quad{.....\quad(2)}$$ <br>
, since nurses going to each patient may not exceed, nor be below than, the neccesities of each patient. <p>
<br>
* Also, we want to specify some constraints (bounds) for the maximal and minimal **work loads** of the nurses:
$$ Z_{max} \geq Load_k \quad \forall k \quad{.....\quad(3)}$$ 
$$ Z_{min} \leq Load_k \quad \forall k \quad{.....\quad(4)}$$ <br>
<p>
<br>
* Also, we want to specify some constraints in order to assign nurses to patients ONLY IF they are **qualified** for the treatments required by the patients:
$$ \sum_{k,i,j} y_{kij} \leq \sum_{k,i,j} skill_{kij} $$
-----------

In [124]:
from pyomo.environ import *
from pyomo.opt import *


model = ConcreteModel()

##########
# let us set the variables for the possible paths:
Distance = [8,6,2] # change the distances from patient/hospital to patient
Needs = [0,1,1] # change the number of nurses needed for each patient
##########
# nurses (A, B) to patients (1, 2)
model.x_A01 = Var(within=Binary)
model.x_A02 = Var(within=Binary)
model.x_A12 = Var(within=Binary)
model.x_A21 = Var(within=Binary)
model.x_B01 = Var(within=Binary)
model.x_B02 = Var(within=Binary)
model.x_B12 = Var(within=Binary)
model.x_B21 = Var(within=Binary)
model.x_C01 = Var(within=Binary)
model.x_C02 = Var(within=Binary)
model.x_C12 = Var(within=Binary)
model.x_C21 = Var(within=Binary)
# maximum work that nurses will do:
work_maxload_A = 40
work_maxload_B = 60
work_maxload_C = 60
work_minload_A = 0
work_minload_B = 0
work_minload_C = 0
# time to treat[i] each patient i:
treat = [0, 5, 1] # python starts indexing from 0 so, for the sake of simplicity, we add a 0 in first place that shall b not used
# total_load = distances between nurses and patients + time for treating each patient
total_load_A01 = Distance[1] + treat[1]
total_load_A02 = Distance[2] + treat[2]
total_load_A21 = Distance[0] + treat[2] + Distance[2] + treat[1]
total_load_A12 = Distance[0] + treat[1] + Distance[1] + treat[2]
total_load_B01 = Distance[1] + treat[1]
total_load_B02 = Distance[2] + treat[2]
total_load_B21 = Distance[0] + treat[2] + Distance[2] + treat[1]
total_load_B12 = Distance[0] + treat[1] + Distance[1] + treat[2]
total_load_C01 = Distance[1] + treat[1]
total_load_C02 = Distance[2] + treat[2]
total_load_C21 = Distance[0] + treat[2] + Distance[2] + treat[1]
total_load_C12 = Distance[0] + treat[1] + Distance[1] + treat[2]
# define if nurse is skilled for the task required by patient i, in skillAi:
skillA1 = 1
skillA2 = 1
skillB1 = 0
skillB2 = 1
skillC1 = 1
skillC2 = 0
# let us minimize the time:
model.obj = Objective(expr= total_load_A01*model.x_A01 + total_load_A02*model.x_A02 + total_load_A21*model.x_A21 + total_load_A12*model.x_A12
                      + total_load_B01*model.x_B01 + total_load_B02*model.x_B02 + total_load_B21*model.x_B21 + total_load_B12*model.x_B12
                      + total_load_C01*model.x_C01 + total_load_C02*model.x_C02 + total_load_C21*model.x_C21 + total_load_C12*model.x_C12
                      , sense=minimize) # minimin cost problem, where cost is total_load

# let us set the constraints based on the needs for each patient:
# nurses going to patient 1:
model.need1 = Constraint(expr= model.x_A01 + model.x_A21 + model.x_A12
                      + model.x_B01 + model.x_B21 + model.x_B12
                      + model.x_C01 + model.x_C21 + model.x_C12 == Needs[1]) # at least Needs[i] nurse(s) will visit patient 1
# nurses going to patient 2:
model.need2 = Constraint(expr= model.x_A02 + model.x_A21 + model.x_A12
                      + model.x_B02 + model.x_B21 + model.x_B12
                      + model.x_C02 + model.x_C21 + model.x_C12 == Needs[2])

# maximum work for each nurse:
model.workloadmaxA = Constraint(expr= total_load_A01*model.x_A01 + total_load_A02*model.x_A02 + total_load_A21*model.x_A21 +
                            total_load_A12*model.x_A12 <= work_maxload_A)
model.workloadmaxB = Constraint(expr= total_load_B01*model.x_B01 + total_load_B02*model.x_B02 + total_load_B21*model.x_B21 +
                            total_load_B12*model.x_B12 <= work_maxload_B)
model.workloadmaxC = Constraint(expr= total_load_C01*model.x_C01 + total_load_C02*model.x_C02 + total_load_C21*model.x_C21 +
                            total_load_C12*model.x_C12 <= work_maxload_C)

# minimum work for each nurse:
model.workloadminA = Constraint(expr= total_load_A01*model.x_A01 + total_load_A02*model.x_A02 + total_load_A21*model.x_A21 +
                             total_load_A12*model.x_A12 >= work_minload_A)
model.workloadminB = Constraint(expr= total_load_B01*model.x_B01 + total_load_B02*model.x_B02 + total_load_B21*model.x_B21 +
                            total_load_B12*model.x_B12 >= work_minload_B)
model.workloadminC = Constraint(expr= total_load_C01*model.x_C01 + total_load_C02*model.x_C02 + total_load_C21*model.x_C21 +
                            total_load_C12*model.x_C12 >= work_minload_C)

# make sure nurse is cualified:
model.qualificationsA01 = Constraint(expr= model.x_A01 <= skillA1)
model.qualificationsA02 = Constraint(expr= model.x_A02 <= skillA2)
model.qualificationsA21 = Constraint(expr= model.x_A21 <= skillA1*skillA2)
model.qualificationsA12 = Constraint(expr= model.x_A12 <= skillA1*skillA2)
model.qualificationsB01 = Constraint(expr= model.x_B01 <= skillB1)
model.qualificationsB21 = Constraint(expr= model.x_B21 <= skillB1*skillB2)
model.qualificationsB12 = Constraint(expr= model.x_B12 <= skillB1*skillB2)
model.qualificationsC01 = Constraint(expr= model.x_C01 <= skillC1)
model.qualificationsC02 = Constraint(expr= model.x_C01 <= skillC2)
model.qualificationsC21 = Constraint(expr= model.x_C21 <= skillC1*skillC2)
model.qualificationsC12 = Constraint(expr= model.x_C12 <= skillC1*skillC2)


## solving:
Solver = SolverFactory('glpk')
results = Solver.solve(model)

## printing results:
print('\n\nx_A01:',value(model.x_A01), '  x_A02:',value(model.x_A02), '  x_A12:',value(model.x_A12), '  x_A21:',value(model.x_A21),
      '\nx_B01:',value(model.x_B01), '  x_B02:',value(model.x_B02), '  x_B12:',value(model.x_B12), '  x_B21:',value(model.x_B21),
     '\nx_C01:',value(model.x_C01), '  x_C02:',value(model.x_C02), '  x_C12:',value(model.x_C12), '  x_C21:',value(model.x_C21))
print('\nthe Minimum Time is:', value(model.obj))



x_A01: 1.0   x_A02: 0.0   x_A12: 0.0   x_A21: 0.0 
x_B01: 0.0   x_B02: 1.0   x_B12: 0.0   x_B21: 0.0 
x_C01: 0.0   x_C02: 0.0   x_C12: 0.0   x_C21: 0.0

the Minimum Time is: 14.0


In [20]:
total_load_B12

12

<font color='red'>**Conclussion**</font> <br> 

We can see how varying any of the parameters alters the final result.
* If you set a nurse as unskilled for a patient, you can see how other will be choosen.
* If you increase enough the distance between some patients, you can see how the programs suggests to choose more and different nurses for each patient (and viceversa).
* If you increase the minimum workload of a nurse, they will ALWAYS be choosen in. at least, on path (and vice versa). 
* However, take care because these modifications can lead to errors if this is not possible. For instance, you can force two nurses to attend patients by setting the minimum workload of them higher. But, if at least one nurse must attend a patient, he/she must be able to go there (skilled). So be careful...

<div class="alert alert-block alert-warning">
    Contact me:</b> <br> 
    <b>$$\rightarrow Pablo \quad Laso\quad Mielgo\quad :)$$<br> $$\rightarrow p.laso.2017@alumnos.urjc.es $$
</div>

<img src="https://www.clipartwiki.com/clipimg/detail/205-2052859_clipart-of-courses-basic-education-and-class-registration.png" alt="Alt text that describes the graphic" title="Title text" />